In [ ]:
import requests
import pandas as pd
import numpy as np
from joblib import load
from sklearn.preprocessing import LabelEncoder
from datetime import datetime, timedelta



model = load("/content/drive/MyDrive/data/Tennis-Prediction.joblib")

url = "https://api.api-tennis.com/tennis/"

now = datetime.now()
date_start = now.strftime('%Y-%m-%d')
date_stop = (now + timedelta(days=1)).strftime('%Y-%m-%d')
print(f"\nFetching tennis data for upcoming matches from {date_start} to {date_stop}")

params = {
    "method": "get_fixtures",
    "APIkey": "bbad081e02e2214b97aa19f1f9151e0e6b84b0caf1e24a5cff55ff663c18382b",
    "date_start": date_start,
    "date_stop": date_stop,
    "timezone": "Europe/London",
    "status": "Not Started"
}

response = requests.get(url, params=params)
data = response.json()
matches = data.get("result", [])

df = pd.DataFrame([{
    "Date": m["event_date"],
    "Time": m["event_time"],
    "Player_1": m["event_first_player"],
    "Player_2": m["event_second_player"],
    "Match": f"{m['event_first_player']} vs {m['event_second_player']}",
    "Level": m["event_type_type"],
    "Odds": np.random.uniform(1.4, 2.5)
} for m in matches if m.get("event_status") == "" and m.get("event_final_result") == "-"])

if df.empty:
    print("❌ No upcoming matches available in the selected range.")
else:
    df["Level"] = LabelEncoder().fit_transform(df["Level"])
    df["log_odds"] = np.log(df["Odds"])
    df["inverse_odds"] = 1 / df["Odds"]
    df["prob_diff"] = 0
    df["risk_index"] = 0

    features = ['Odds', 'Level', 'log_odds', 'inverse_odds', 'prob_diff', 'risk_index']
    df["model_probability"] = model.predict_proba(df[features])[:, 1]
    df["model_expected_value"] = (df["model_probability"] * (df["Odds"] - 1)) - (1 - df["model_probability"])
    df["model_expected_value_percent"] = df["model_expected_value"] * 100
    df["prob_diff"] = df["model_probability"] - df["inverse_odds"]
    df["risk_index"] = (df["model_probability"] * df["Odds"]) - 1

    df["prediction"] = np.where(
        df["model_probability"] >= 0.5,
        df["Player_1"] + " to win match",
        df["Player_2"] + " to win match"
    )

    value_bets = df[
        (df["model_probability"] >= 0.6) &
        (df["model_expected_value"] >= 0.15)
    ].copy()

    print("📅 High-Value Upcoming Matches:")
    if value_bets.empty:
        print("⚠️ No high-value bets found at the moment.")
    else:
        for _, row in value_bets.iterrows():
            print(f"🗓 {row['Date']} | {row['Match']}")
            print(f"🎯 Prediction: {row['prediction']}")
            print(f"📈 Prob: {row['model_probability']:.2%} | EV: {row['model_expected_value_percent']:.2f}% | Odds: {row['Odds']:.2f}")
            print("—" * 40)

        print(f"\n✅ Number of high-value bets found: {len(value_bets)}")



Fetching tennis data for upcoming matches from 2025-06-23 to 2025-06-24
📅 High-Value Upcoming Matches:
⚠️ No high-value bets found at the moment.


In [ ]:
output_path = "/content/drive/MyDrive/sav/high_value_upcoming_bets.csv"
value_bets[[
    "Date", "Match", "Odds", "model_probability",
    "model_expected_value_percent", "prediction"
]].to_csv(output_path, index=False)

print(f"\n📥 Results have been saved to file: {output_path}")




📥 Results have been saved to file: /content/drive/MyDrive/sav/high_value_upcoming_bets.csv


In [ ]:
from datetime import datetime, timedelta
now = datetime.now()
date_start = now.strftime('%Y-%m-%d')
date_stop = (now + timedelta(days=1)).strftime('%Y-%m-%d')
print(f"\nFetching tennis data for upcoming matches from {date_start} to {date_stop}")



Fetching tennis data for upcoming matches from 2025-06-23 to 2025-06-24
